In [1]:
import pandas as pd
import json
import re

In [2]:
col = ["indicateur", "period_date", "valeur", "mesure", "short_indic", "dep", "reg"]
df = pd.read_csv("/home/sgpr/Projets/Widget_first_draft/dashboard_widgets/donnee_jupiter/pp_dep.csv", usecols=col, sep=";")
df['period_date'] = pd.to_datetime(df['period_date'])

In [3]:
df.columns

Index(['indicateur', 'period_date', 'valeur', 'mesure', 'short_indic', 'dep',
       'reg'],
      dtype='object')

In [4]:
indicList = df.short_indic.unique()
dfList = []
for indic in indicList:
    dfList.append(df.loc[df.short_indic == indic])

In [5]:
# create objects for json
listObject = []

def evolVal(valI, valE):
    if (valE == 0):
        return 0
    elif(valI != valE):
        return valI - valE
    else:
        return 0

def evolPercent(ev, val):
    round((ev / val)*100,2)


for dff in dfList:
    data = {}
    
    data["nom"] = dff["indicateur"].iloc[0]
    data["unite"] = dff["short_indic"].iloc[0]
    
    # FRANCE
    most_recent_date = dff['period_date'].max()
    dfDate = dff.loc[dff.period_date == most_recent_date]
    sumValue = dfDate["valeur"].sum()
    
    #calcul evolution % :
    dfevol = pd.DataFrame()
    dfevol = dff.copy() 
    dfevol.drop(dfevol.loc[dfevol['period_date'] == most_recent_date].index, inplace=True)
    most_recent_date2 = dfevol['period_date'].max()
    dfDatee = dfevol.loc[dfevol.period_date == most_recent_date2]
    sumValue2 = dfDatee["valeur"].sum()
    evol = evolVal(sumValue, sumValue2)
    evol_percent = evolPercent(evol, sumValue2)
    
    #fabrication json france :
    france = {}
    france["last_value"] = sumValue
    france["last_date"] = str(most_recent_date) #.to_pydatetime()
    france["evol"] = evol
    france["evol_percentage"] = evol_percent
    france["evol_color"] = "red"
    france["level"] = "nat"
    france["code_level"] = "fra"
    dates = dff.sort_values(by="period_date").period_date.unique()
    values = []
    for date in dates:
        value = dff.loc[dff.period_date == date]
        values += [{"date": date.astype(str), "value": value["valeur"].sum()}]
        
    france["values"] = values
    c = []
    c.append(france)
    data["france"] = c
    
    # REGION
    
    regionList = []
    most_recent_date = ""
    reg = []
    reg = dff.reg.unique()
    for region in reg:
        dfreg = dff.loc[dff.reg == region]
        most_recent_date = dfreg['period_date'].max()
        sumValueReg = dfreg["valeur"].sum()
        
        #calcul evolution % :
        dfevol = pd.DataFrame()
        dfevol = dfreg.copy() 
        dfevol.drop(dfevol.loc[dfevol['period_date'] == most_recent_date].index, inplace=True)
        most_recent_date2 = dfevol['period_date'].max()
        dfDatee = dfevol.loc[dfevol.period_date == most_recent_date2]
        sumValue2 = dfDatee["valeur"].sum()
        evol = 0
        evol_percent = 0
        evol = evolVal(sumValue, sumValue2)
        evol_percent = evolPercent(evol, sumValue2)
        
        #fabrication json region :
        regionDict = {}
        regionDict["last_value"] = sumValueReg
        regionDict["last_date"] = str(most_recent_date) #.to_pydatetime()
        regionDict["evol"] = evol
        regionDict["evol_percentage"] = evol_percent
        regionDict["evol_color"] = "red"
        regionDict["level"] = "reg"
        regionDict["code_level"] = region
        
        dates = dfreg.sort_values(by="period_date").period_date.unique()
        values = []
        for date in dates:
            value = dfreg.loc[dfreg.period_date == date]
            values += [{"date": date.astype(str), "value": value["valeur"].sum()}]
        
        regionDict["values"] = values
        regionList.append(regionDict)
        
    data["regions"] = regionList
    
    # DEPARTEMENTS
    
    departementList = []
    most_recent_date = ""
    dep = []
    dep = dff.dep.unique()
    for dpt in dep:
        dfdep = dff.loc[dff.reg == dpt]
        most_recent_date = dfdep['period_date'].max()
        sumValueDep = dfdep["valeur"].sum()
        
        #calcul evolution % :
        dfevol = pd.DataFrame()
        dfevol = dfdep.copy() 
        dfevol.drop(dfevol.loc[dfevol['period_date'] == most_recent_date].index, inplace=True)
        most_recent_date2 = dfevol['period_date'].max()
        dfDatee = dfevol.loc[dfevol.period_date == most_recent_date2]
        sumValue2 = dfDatee["valeur"].sum()
        evol = 0
        evol_percent = 0
        evol = evolVal(sumValue, sumValue2)
        evol_percent = evolPercent(evol, sumValue2)
        
        #fabrication json departement :
        depDict = {}
        depDict["last_value"] = sumValueDep
        depDict["last_date"] = str(most_recent_date)
        depDict["evol"] = evol
        depDict["evol_percentage"] = evol_percent
        depDict["evol_color"] = "red"
        depDict["level"] = "reg"
        depDict["code_level"] = dpt
        dfdep = dfdep.rename(columns={"period_date": "date", "valeur": "value"})
        depDict["values"] = json.loads("[" + dfdep[["date", "value"]].sort_values(by="date").to_json(orient='records', date_format="iso", date_unit="s")[1:-1] + "]")
        departementList.append(depDict)
        
    data["departements"] = departementList
    
    listObject.append(data)

    

<ipython-input-5-66815b72eea2>:13: RuntimeWarning: invalid value encountered in double_scalars
  round((ev / val)*100,2)


In [6]:

for jsonObj in listObject:
    
    list_newfilename = []

    name = jsonObj["nom"]
    newfilename = name

    newfilename = re.sub(' ', '_', newfilename)
    newfilename = re.sub('/', '&', newfilename)
    newfilename = re.sub('"', '', newfilename)

    path = "/home/sgpr/Projets/Widget_first_draft/dashboard_widgets/donnee_jupiter/json/" + newfilename +".json"
    print(newfilename)
    with open(path, "w", encoding="utf8") as f:
        json.dump(jsonObj, f, ensure_ascii=False)
    path = "/home/sgpr/Projets/Widget_first_draft/dashboard_widgets/public/json/" + newfilename +".json"
    with open(path, "w", encoding="utf8") as f:
        json.dump(jsonObj, f, ensure_ascii=False)
    
    with open('/home/sgpr/Projets/Widget_first_draft/dashboard_widgets/public/list_jsonfiles.txt', 'a') as f:
        f.write("%s\n" % newfilename)
    



Nombre_de_repas_servis_dans_les_restaurants_universitaires_au_tarif_à_1€_-_RRU
Nombre_d’entrées_en_service_civique_-_SCI
Nombre_de_PME&TPE_-_EEI
Nombre_d’entreprises_ayant_reçu_l’aide_-_EEI
Nombre_de_projets_soutenus_-_ETH
Montant_total_de_CO2_évité_-_ETH
Montant_cumulé_de_l’investissement_total_ainsi_déclenché_-_CBC
Nombre_de_logements_subventionnés_-_RER
Emissions_de_gaz_à_effet_de_serre_évitées_sur_la_durée_de_vie_des_équipements_-_PEE
Nombre_de_centres_de_tri_modernisés_-_MCT
Nombre_d’installations_CSR_soutenues_-_MCT
Tonnage_annuel_de_biodéchets_soutenu_-_MCT
Nombre_de_projets_de_tourisme_durable_financés_-_TDU
Montant_de_l’investissement_total_déclenché_- _ETE
Nombre_d’entreprises_bénéficiaires_-_ETE
Nombre_de_bonus_octroyés_à_des_véhicules_électriques_et_hybrides_rechargeables_-_BOE
Quantité_de_matières_plastiques_évitées_ou_dont_le_recyclage_ou_l’intégration_a_été_soutenue_-_IRR
Nombre_de_primes_à_la_conversion_-_DVP
Montant_total_des_primes_versées_-_MPR
Montant_total_des_trav